In [1]:
import pika

連線可分為三層:  
1. credential: 如果有帳密就在這部分進行設定
2. parameters: 負責連線資訊，包括連線server的ip和連線帳號。需要加上heartbeat是為告知server目前連線還存在，否則Server預設5分鐘就會斷線。
3. connection: 決定連線的Session。  
連線後需要創建channel，該channel中可以放入訊息的序列。

In [3]:
connection_credential = pika.PlainCredentials(username='criuser', password='criuser')

connection_parameters = pika.ConnectionParameters(host='xxx.xxx.xxx',
                                                  port=5672,
                                                  credentials=connection_credential,
                                                  heartbeat=0,
                                                  blocked_connection_timeout=86400)

server = pika.BlockingConnection(connection_parameters)
channel = server.channel()

channel中可以放置不同的queue，並且針對不同的queue可以做message publushing

In [3]:
queue_stat = channel.queue_declare(queue='first_que') #create a queue
queue_stat = channel.queue_declare(queue='second_que') #create another queue

rabbitmq中，交換資料的主要仰賴exchange這物件。exchanger總共有四種類型:
* direct: 會比對ring是否相符
* topic:跟direct很像，但是只要符合某些關鍵字就發送，不會像direct會嚴格比對
* headers
* fanout: 有甚麼就丟甚麼出去，大家喜歡怎麼收就全收下來  
  先使用fanout模式來進行Subscribe功能

[參考資料](https://www.rabbitmq.com/tutorials/tutorial-three-python.html)

In [31]:
#測試subscribe的交換方法。這個exchanger最好在publisher跟subscriptor都做一次。
channel.exchange_declare(exchange="sub", exchange_type='fanout') #建構exchange

#清理queue
channel.queue_purge('first_que')
channel.queue_purge('second_que')

#把會訂閱的queue綁定。只需要在subscriptor上做就可。
channel.queue_bind(exchange='sub', queue='first_que')
channel.queue_bind(exchange='sub', queue='second_que') #也能用 queue=queue_stat.method.queue 。因為最後queue_stat是指到second_que。

#發布消息。在publisher上做。
channel.basic_publish(
    exchange='sub', #給定這個exchanger名稱
    routing_key='', 
    body="today is a good day"
)

In [32]:
# 取得消息。在subscriptor上做。
## method1: 使用內建迴圈方法持續讀取。
# def msg_callback(channel, method, properties, body):
#     print("msg: {}".format(body))
    
# res = channel.basic_consume(queue='second_que', on_message_callback=msg_callback)
# channel.start_consuming() #這邊會有一個無限迴圈，不斷進行rabbitMQ的讀取

##method2
res = channel.basic_get(queue='first_que')
print(res[2])
res = channel.basic_get(queue='second_que')
print(res[0])
print(res[0].message_count)


b'today is a good day'
<Basic.GetOk(['delivery_tag=16', 'exchange=sub', 'message_count=0', 'redelivered=False', 'routing_key='])>
0
